In [1]:
import dvach
import os
import pandas as pd
import re
from tqdm import tqdm
from time import sleep
# Declare the board
board = dvach.Board('b')

In [3]:
# Download threads
board.update_threads()

In [4]:
len(board.threads)

208

In [5]:
all_boards = []

with open("all_boards.txt", "r") as f:
    all_boards = [board for board in f.read().split("\n") if board != ""]

In [6]:
all_boards

['d',
 'b',
 'o',
 'soc',
 'media',
 'r',
 'api',
 'rf',
 'po',
 'news',
 'hry',
 'au',
 'bi',
 'biz',
 'bo',
 'c',
 'em',
 'fa',
 'fiz',
 'fl',
 'ftb',
 'hh',
 'hi',
 'me',
 'mg',
 'mlp',
 'mo',
 'mov',
 'mu',
 'ne',
 'psy',
 're',
 'sci',
 'sf',
 'sn',
 'sp',
 'spc',
 'tv',
 'un',
 'w',
 'wh',
 'wm',
 'wp',
 'zog']

In [7]:
# post_id_answer_id = pd.DataFrame(columns=["post_id", "answer_id"])
# post_id_post_text = pd.DataFrame(columns=["post_id", "post_text"])

In [8]:
# Function to extract all the numbers from the given string
def getNumbers(str):
    array = re.findall(r'[0-9]+', str)
    return array

In [10]:
posts_thread_sleep = 50

for board_id in tqdm(all_boards[6:]):
    post_id_post_text_arr = []
    post_id_answer_id_arr = []
    
    board = dvach.Board(board_id)
    board.update_threads()
    for thread in board.threads.values():
        try:
            thread.update_posts()
        except KeyError as er:
#             print(f"Thread {thread.num} doesn't have replies")
            continue
        except:
            continue
#         print(f"Thread {thread.num} contain {len(thread.posts)} posts")
        open_post = thread.posts[0]
        post_id_post_text_arr.append([int(open_post.num), open_post.comment])
        for post in thread.posts[1:]:
            # список постов, на которые отвечает текущий пост
            curr_post_id = post.num.__str__()
            reply_ids = getNumbers(post.comment)

            # отфильтруем ошибочные числа, оставив только те, которые являются id'шниками
            reply_ids = [post_id for post_id in reply_ids if len(post_id) == len(curr_post_id)]

            # если пост не является ответом на другой пост не берём
            if len(reply_ids) == 0:
                continue

            # отсекаем id'шники реплаи и длину последнего айдишника, оставляя только сам текст
            text = post.comment.split(">>")[-1][len(reply_ids[-1]):]
            text = text[5:] if text.startswith(" (OP)") else text

            post_id_post_text_arr.append([int(curr_post_id), text])
            for post_id in reply_ids:
                post_id_answer_id_arr.append([int(post_id), int(curr_post_id)])
            
            posts_thread_sleep -= 1
            if posts_thread_sleep == 0:
                sleep(5)
                posts_thread_sleep = 100
    
    post_id_answer_id = pd.DataFrame(post_id_answer_id_arr, columns=["post_id", "answer_id"])
    post_id_post_text = pd.DataFrame(post_id_post_text_arr, columns=["post_id", "post_text"])
    post_id_answer_id.to_csv(f"../threads/post_id_answer_id_{board_id}.csv", index=False)
    post_id_post_text.to_csv(f"../threads/post_id_text_{board_id}.csv", index=False)

    # спим, чтобы не прервать соединение из-за частых запросов
    sleep(10)

100%|███████████████████████████████████████████████████████████████████████████████| 38/38 [5:48:04<00:00, 549.58s/it]
